In [10]:
import pandas as pd
import requests as r
import time
import json
from mailjet_rest import Client
import os


In [11]:
inputs = pd.read_csv('server_input.csv')
patient = 'Susan Foster'
patient_email_addr = 'susan@foster.com'

def authenticate_mailjet():
    if os.getenv('MAILJET_PUBLIC_KEY') is not None \
    and os.getenv('MAILJET_SECRET_KEY') is not None:
        api_key = os.environ['MAILJET_PUBLIC_KEY']
        api_secret = os.environ['MAILJET_SECRET_KEY']
        mailjet = Client(auth=(api_key, api_secret), version='v3')
        return mailjet
    print('Please set MAILJET_PUBLIC_KEY and MAILJET_SECRET_KEY env variable before send email')
    return None

In [12]:
def send_email():
    mailjet = authenticate_mailjet()
    email = {
        'FromName': 'Fall Detection App',
        'FromEmail': 'sfu73318@gmail.com',
        'Subject': patient+' is in danger',
        'Text-Part': 'Hey! We suspect,'+patient+' just encountered a fall, please check.',
        'Recipients': [{'Email': patient_email_addr}]
    }
    if mailjet is not None:
        mailjet.send.create(email)

In [13]:
i = 0
while (i < 100):
    i += 1
    time.sleep(1)
    request_data = json.dumps({
        "features":list(inputs.sample(n = 1).values.tolist())
    })
    headers = {'Content-Type': 'application/json'}
    response = r.post('https://mdrmuhaimin.me/fall-detection/api/v1/predict', headers=headers, data=request_data).json()
    prediction = response['prediction']
    print(request_data, prediction)
    if 1 in prediction:
        print('Fall detected for patient ', patient, 'Sending email to ', patient_email_addr)
        send_email()
        break

{"features": [[3.318710718749999, 4.092538501198874, 8.075638175272099, 5.302042366960035, -3.3596941875000006, 4.185834528197837, 2.301334319238006, 6.14160765625]]} [1]
Fall detected for patient  Susan Foster Sending email to  susan@foster.com
Please set MAILJET_PUBLIC_KEY and MAILJET_SECRET_KEY env variable before send email


NameError: name 'mailjet' is not defined